# Fine-Tuning an Embedding Model
***
#### This exercise won't cost you anything except some time...

#### It is recommended to run this specific section either locally on your machine or on Google Colab. As a proxy, it will take less than 10 minutes to run in a Macbook Air M1.

Aside from switching out your emebdding model to improve your retrieval results, you could also try fine tuning your embedding model (or better yet, switch out your model and then fine tune the new one...👊).  For the longest time, the problem with fine-tuning sentence embedding models was the lack of access to high quality training data.  Generative LLMs can save you days/weeks of time, depending on how large of a dataset you want to create, by automating the process of generating high quality query/context pairs.  In this section we'll go over the step-by-step process of fine-tuning our `all-MiniLM-L6-v2` embedding model from a pre-generated training dataset consisting of only 300 question-context pairs, and then comparing it's retrieval results to our baseline retrieval scores.  I highly encourage trying this method out, I saw a 10+ point jump in `vector_hit_rate` after fine-tuning the baseline model.

### Fine-tune Walkthrough

1. Get baseline retrieval scores (vector Hit Rate, MRR, and total misses) using out-of-the-box baseline model.  You won't know objectively if fine-tuning had any effect if you don't measure the baseline results first.  I know this goes without saying it, but practitioners sometimes want to jump straight into model improvement without first considering their starting point.
2. Collect a training and validation dataset.  This step has already been completed for you, courtesy of `gpt-3.5-turbo`.  LlamaIndex has a great out-of-the-box solution for generating query/context embedding pairs, but it isn't exactly plug and play, so I had to rewrite the function to achieve comptability for our course.  The training dataset consists of queries generated by the LLM that can be answered from the associated context (text chunk).  These pairs were generated using a prompt specifically written for the Impact Theory corpus so the training and validation data (for the most part) are high quality and contextually relevant. 
3. Instantiate a `SentenceTransformersFinetuneEngine` Class written by LlamaIndex which does a great job of abstracting away most of the details invovled in fine-tuning a Sentence Transformer model.
4. Fit the model and set a path where the new model will reside.  I creaed a `models/` directory in the course repo, and included the directory in the `.gitignore` file so that models aren't being pushed with every commit.
5. Create a new dataset (as you learned in Notebook 1) but this time create the embeddings using the new fine-tuned model.
6. Create a new index on Weaviate using the new dataset you just created.
7. Run the `retrieval_evaluation` function again, but this time instantiate your Weaviate client with the new fine-tuned model, but hold all other parameters constant (i.e. don't change any other parameter from the baseline run).
8. Compare the fine-tuned retrieval results to the baseline results 🥳

### Import Training + Valid datasets

In [ ]:
training_path = './data/training_data_300.json'
valid_path = './data/validation_data_100.json'

In [ ]:
training_set = EmbeddingQAFinetuneDataset.from_json(training_path)
valid_set = EmbeddingQAFinetuneDataset.from_json(valid_path)
num_training_examples = len(training_set.queries)
num_valid_examples = len(valid_set.queries)
print(f'# Training Samples: {num_training_examples}\n# Validation Samples: {num_valid_examples}')

In [ ]:
#always a good idea to name your fine-tuned so that you can easily identify it,
#especially if you plan on doing multiple training runs with different params
#also probably a good idea to include the # of training samples you are using in the name

model_id = client.model_name_or_path
model_ext = model_id.split('/')[1]
models_dir = './models'
if not os.path.exists('./models'):
    os.makedirs('./models') 
else:
    print(f'{models_dir} already exists')
ft_model_name = f'finetuned-{model_ext}-{num_training_examples}'
model_outpath = os.path.join(models_dir, ft_model_name)

print(f'Model ID: {model_id}')
print(f'Model Outpath: {model_outpath}')

### Instantiate your Fine-tune engine

In [ ]:
from llama_index.finetuning import SentenceTransformersFinetuneEngine

finetune_engine = SentenceTransformersFinetuneEngine(
    training_set,
    batch_size=32,
    model_id=model_id,
    model_output_path=model_outpath,
    val_dataset=valid_set,
    epochs=10
)

### Fit the embedding model

In [1]:
# finetune_engine.finetune()

The `finetune` method will automatically generate the model directory using the `model_output_path` that you define.  Inside the directory will be a copy of the model itself (`pytorch_model.bin`) along with all the other files it needs.  Also in that folder, assuming you provide a `val_dataset` will be an evaluation report in the `eval` directory.  The evaluation report contains several IR metrics that may or may not be useful to you, but it does allow you to compare score improvements with each training epoch.  The new fine-tuned model is loaded through the `SentenceTransformer` class just like any other HuggingFace repo model. 